In this notebook we read in the Master XML file made using NapaTrackMater and create N, Delta times Attribute dimensional vectors. N being the number of tracks present in the chosen region, R, Delta being the chosen time interval (t_minus + t_plus) {t - t_minus, t + t_plus} and Attributes being the morphological and the dynamic properties associated with cells in the tracks. We concatenate the Attribute componenets over the chosen time interval to create a Delta times Attribute dimensional vector and create a pandas dataframe with these vectors for further analysis.

In [ ]:
!pip install napatrackmater
!pip install git+https://github.com/kapoorlab/cellshape-helper
!pip install git+https://github.com/kapoorlab/cellshape-cloud
!pip install git+https://github.com/kapoorlab/cellshape-cluster    

In [1]:
%gui qt5
from napatrackmater.Trackvector import TrackVector
from pathlib import Path
from ipywidgets import interactive, widgets
from IPython.display import display
import napari 
from tifffile import imread
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

C:\Users\rando\anaconda3\envs\naparienv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\rando\anaconda3\envs\naparienv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\rando\anaconda3\envs\naparienv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
master_xml_path = Path('C:/Users/rando/Downloads/Mari_project/master_test_tracks.xml')
spot_csv_path = Path('C:/Users/rando/Downloads/Mari_project/test_spots.csv')
track_csv_path = Path('C:/Users/rando/Downloads/Mari_project/test_tracks.csv')
edges_csv_path = Path('C:/Users/rando/Downloads/Mari_project/test_edges.csv')
image = imread('C:/Users/rando/Downloads/Mari_project/gt/rawk.tif')
show_tracks = False



In [3]:
if show_tracks:
  viewer = napari.Viewer()
else:
    viewer = None
track_vectors = TrackVector(viewer,image,master_xml_path,spot_csv_path, track_csv_path, edges_csv_path, show_tracks = show_tracks)

Reading Master XML
Iterating over spots in frame
Iterating over tracks 540
getting attributes
obtianed spot attributes
obtained track attributes
obtained edge attributes


In [4]:
def track_setter(deltat, deltax, deltay):
    track_vectors.t_minus = deltat[0]
    track_vectors.t_plus = deltat[-1]
    
    track_vectors.x_start = deltax[0]
    track_vectors.x_end = deltax[-1]
    
    track_vectors.y_start = deltay[0]
    track_vectors.y_end = deltay[-1]
    

In [5]:
track_vector_widgets = interactive(track_setter, deltat = widgets.IntRangeSlider(
    value=[track_vectors.tstart, track_vectors.tend],
    min= track_vectors.tstart,
    max=track_vectors.tend,
    step=1,
    description='Delta Time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltax = widgets.IntRangeSlider(
    value=[track_vectors.xmin, track_vectors.xmax],
    min= track_vectors.xmin,
    max=track_vectors.xmax,
    step=1,
    description='Delta X',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
), 
    deltay = widgets.IntRangeSlider(
    value=[track_vectors.ymin, track_vectors.ymax],
    min= track_vectors.ymin,
    max=track_vectors.ymax,
    step=1,
    description='Delta Y',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
),                               
                                  
                                  )

track_vector_widgets

interactive(children=(IntRangeSlider(value=(0, 265), continuous_update=False, description='Delta Time', max=26…

In [6]:
track_vectors._interactive_function()

returning shape and dynamic vectors as list 573


In [7]:
current_shape_dynamic_vectors = track_vectors.current_shape_dynamic_vectors

In [8]:
global_shape_dynamic_dataframe = []
for i in range(len(current_shape_dynamic_vectors)):
   
   vector_list = list(zip(current_shape_dynamic_vectors[i]))
   data_frame_list = np.transpose(np.asarray([vector_list[i] for i in range(len(vector_list))])[:,0,:]) 
   
   shape_dynamic_dataframe = pd.DataFrame(data_frame_list, columns =['Track ID', 't', 'z', 'y', 'x', 'Dividing', 'Number_Dividing',  'Radius', 'Volume', 'Eccentricity Comp First', 'Eccentricity Comp Second', 'Surface Area', 'Cluster Class', 'Speed', 'Motion_Angle', 'Acceleration', 'Distance_Cell_mask', 'Radial_Angle', 'Cell_Axis_Mask'])
   if len(global_shape_dynamic_dataframe) == 0:
        global_shape_dynamic_dataframe = shape_dynamic_dataframe
   else:
        global_shape_dynamic_dataframe = pd.concat([global_shape_dynamic_dataframe, shape_dynamic_dataframe],ignore_index=True)

global_shape_dynamic_dataframe = global_shape_dynamic_dataframe.set_index('Track ID')
global_shape_dynamic_dataframe = global_shape_dynamic_dataframe.sort_values(by=['Track ID'])
global_shape_dynamic_dataframe = global_shape_dynamic_dataframe.sort_values(by=['t'])


In [9]:
global_shape_dynamic_dataframe.head()

,t,z,y,x,Dividing,Number_Dividing,Radius,Volume,Eccentricity Comp First,Eccentricity Comp Second,Surface Area,Cluster Class,Speed,Motion_Angle,Acceleration,Distance_Cell_mask,Radial_Angle,Cell_Axis_Mask
Track ID,,,,,,,,,,,,,,,,,,
1253900.0,0.0,6.92663,91.203804,146.396739,0.0,0.0,7.478581,1840.0,2.813315,2.56962,2.149039,1.0,0.0,0.0,0.0,27.50802,13.231419,44.46238
1853900.0,0.0,10.087282,51.19202,13.51995,0.0,0.0,5.670283,802.0,2.427029,1.901139,1.143225,2.0,0.0,0.0,0.0,3.66569,37.776089,43.020017
1153900.0,0.0,8.532475,152.338848,136.094975,0.0,0.0,7.185439,1632.0,2.696996,1.850037,1.72933,0.0,0.0,0.0,0.0,23.715024,4.151727,45.859234
1653900.0,0.0,10.380299,53.546135,100.665212,0.0,0.0,7.144109,1604.0,2.477043,1.932991,0.804186,2.0,0.0,0.0,0.0,29.813412,19.759035,44.182668
1753900.0,0.0,5.316428,27.571114,137.785006,0.0,0.0,5.907663,907.0,2.351466,2.285738,0.895669,2.0,0.0,0.0,0.0,13.127069,33.595516,44.290085


In [10]:
global_shape_dynamic_dataframe.tail()

,t,z,y,x,Dividing,Number_Dividing,Radius,Volume,Eccentricity Comp First,Eccentricity Comp Second,Surface Area,Cluster Class,Speed,Motion_Angle,Acceleration,Distance_Cell_mask,Radial_Angle,Cell_Axis_Mask
Track ID,,,,,,,,,,,,,,,,,,
3.745390e+07,265.0,8.264368,38.700627,114.624869,0.0,0.0,7.577523,1914.0,3.810634,2.798831,2.486187,1.0,0.006807,74.672719,0.001537,19.131306,26.988626,44.843325
5.235390e+07,265.0,8.529412,17.54902,14.228291,0.0,0.0,6.872625,1428.0,2.405382,2.323742,1.759459,0.0,0.003628,68.516794,0.005013,2.990509,43.614302,46.867616
5.255390e+07,265.0,8.774617,85.358862,34.573304,0.0,0.0,4.700947,457.0,3.27687,2.932062,0.830322,2.0,0.005257,14.66509,0.008836,19.159117,25.181534,44.440453
4.785391e+09,265.0,6.535067,67.739953,158.988968,1.0,1.0,6.607449,1269.0,3.607415,3.295661,2.034214,0.0,0.009817,18.963628,0.002341,19.707849,21.91843,44.233236
4.905390e+07,265.0,7.955789,184.469474,28.709474,0.0,0.0,6.867809,1425.0,3.096002,2.620844,1.842195,0.0,0.003867,59.024636,0.005039,1.850751,36.075096,46.117068


In [29]:
base_columns = global_shape_dynamic_dataframe.iloc[:, : 4]
label_columns = global_shape_dynamic_dataframe.iloc[:, 4 : 6 ]
spatial_columns = global_shape_dynamic_dataframe.iloc[:, [1,2,3,6,7,8,9,10,11]] 
dynamic_columns = global_shape_dynamic_dataframe.iloc[:, 12:]
